# コンペティション課題2

## 課題

CNNを用いて、[Tensorflow Flower Dataset](https://www.tensorflow.org/tutorials/load_data/images)のクラス分類を行います。

## 目標値

Accuracy: 80%  
（これはあくまで「目標値」であるため、達成できなかったからといって不合格となったり、著しく成績が損なわれることはありません）

## ルール

- 「修正しないでください」とあるセルを、修正しないでください。
- モデルのアーキテクチャは自由です。講義で扱ったモデル以外でも構いません。
- 訓練データはx_train, t_train、テストデータはx_testで与えられます。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベルによって表してください。
- 以下のセルで定義されているx_train, t_train以外の学習データは使わないでください。

## 提出方法
- 1つのファイルを提出していただきます。
  1. テストデータ`test_iter`に対する予測ラベルを`submission1_pred.csv`として保存・ダウンロードし、Homeworkタブから**Day1 Pred (.csv)**を選択して提出してください。
  2. それとは別に、最終提出に対応するノートブックをわかるようにiLect上に置いておいてください。


- 成績優秀者には、次回講義にて取り組みの発表をお願いいたします。

## 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します。

## LeaderBoard

- コンペティション期間中のLeaderBoardは提出されたcsvファイルのうち50%を使って計算されます。コンペティション終了時には提出されたcsvファイルのうち、コンペティション期間中のLeaderBoard計算に使われなかったもう半分のデータがスコア計算に使用されます。
- このため、コンペ中の順位とコンペ終了後にLeaderBoardが更新された後の順位やスコアが食い違うことがあります。

In [1]:
!pip install timm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## データの読み込み (仮のコード・そもそもtestデータを作っていない）

- このセルは修正しないでください。
- 誤って修正した場合は、元ファイルをコピーし直してください。

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [3]:
class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train, transform):
        self.x_train = x_train
        self.t_train = t_train.astype('int32')
        self.transform = transform

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_train[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        label = torch.tensor(self.t_train[idx], dtype=torch.long)
        if self.transform:
            data = self.transform(data)
        return data, label


class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test, transform):
        self.x_test = x_test
        self.transform = transform

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        data = torch.tensor(self.x_test[idx], dtype=torch.float)
        data = torch.transpose(data,0,2)
        data = torch.transpose(data,1,2)
        if self.transform:
            data = self.transform(data)
        return data

### 畳み込みニューラルネットワーク(CNN)の実装

In [4]:
import os
import random

import numpy as np
import torch

def set_seed(seed: int = 0):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
set_seed(42)

In [6]:
# result_df = pd.DataFrame(columns=['image_height', 'image_width', 'transform_train', 'transform_test',
#        'val_ratio', 'batch_size', 'model', 'optimizer', 'loss_function',
#        'learning_rate', 'num_epoch'])

In [7]:
result_df = pd.read_csv('homework2.csv', index_col=0)

In [8]:
result_df

,image_height,image_width,transform_train,transform_test,val_ratio,batch_size,model,optimizer,loss_function,learning_rate,...,min_valid_loss__epoch,min_valid_loss__train_loss,min_valid_loss__train_acc,min_valid_loss__valid_loss,min_valid_loss__valid_acc,max_valid_acc__epoch,max_valid_acc__train_loss,max_valid_acc__train_acc,max_valid_acc__valid_loss,max_valid_acc__valid_acc
0,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.731455,0.720821,0.804275,0.726225,4,0.731455,0.720821,0.804275,0.726225
1,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.571860,0.778458,0.762222,0.716138,4,0.571860,0.778458,0.762222,0.716138
2,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.670148,0.737752,0.826592,0.691643,4,0.670148,0.737752,0.826592,0.691643
3,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.517022,0.805115,0.626215,0.766571,4,0.517022,0.805115,0.626215,0.766571
4,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.532323,0.801513,0.561638,0.793948,4,0.532323,0.801513,0.561638,0.793948
5,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,3,0.522402,0.804395,0.606840,0.795389,3,0.522402,0.804395,0.606840,0.795389
6,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,64,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.517843,0.806196,0.676056,0.740634,2,0.652513,0.761888,0.716556,0.746398
7,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.787554,0.702089,0.791451,0.713256,4,0.787554,0.702089,0.791451,0.713256
8,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,8,0.695362,0.737032,0.745436,0.717579,8,0.695362,0.737032,0.745436,0.717579
9,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,7,0.587865,0.780620,0.595602,0.779539,8,0.583386,0.777017,0.625318,0.785303


In [9]:
import timm

In [10]:
# timm.list_models()

In [11]:
# %%writefile /root/userspace/submission2_code.py
# 上記のコメントアウトを外し'%%'以下を実行すると、提出コードが出力できます


# 精度向上ポイント: Data Augmentation手法
# transform_train =  transforms.Compose([])
# transform_test = transforms.Compose([])

# data augmentationとデータ整形
height = 224
width = 224
batch_size = 1
transform_train = transforms.Compose(
    [
         transforms.ToPILImage(),
         transforms.RandomHorizontalFlip(p = 0.5),
#          transforms.RandomVerticalFlip(p=0.5),
         transforms.RandomRotation(degrees=30,expand=True),
         transforms.Resize((height,width)), # 画像によってサイズが変わってはならない。
#          transforms.RandomResizedCrop((height, width)),
         transforms.ToTensor()
    ]
)
transform_test = transforms.Compose(
[
    transforms.ToPILImage(),
    transforms.Resize((height,width)),
    transforms.ToTensor()
]
)
x_train = np.load('/root/userspace/public/day2/homework2/data/x_train.npy')
t_train = np.load('/root/userspace/public/day2/homework2/data/t_train.npy')
    
# テストデータ
x_test = np.load('/root/userspace/public/day2/homework2/data/x_test.npy')

trainval_data = train_dataset(x_train, t_train, transform_train)
test_data = test_dataset(x_test,transform_test)
val_ratio = 0.2
val_size = int(len(trainval_data) * val_ratio)
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

# 精度向上ポイント: バッチサイズ
batch_size = 32

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

# rng = np.random.RandomState(1234)
# random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
base_dict = {
    'image_height': height,
    'image_width': width,
    'transform_train': transform_train,
    'transform_test': transform_test,
    'val_ratio': val_ratio,
    'batch_size': batch_size
}

'vit_base_patch32_224'  
'efficientnet_b2'  
'tf_efficientnet_b2_ns'  
'resnest50d'  
'rexnet_130'  


現状提出はバッチ16のエポック数10モデル```'efficientnet_b3'```

更新  
```'vit_base_patch32_224'``` ```result_df.loc[39]```


In [ ]:
# timm.list_models('*efficientnet*')

'repvgg'  
'resnest'  
'rexnet'  
'vit'  



In [14]:
model_tuned = timm.create_model('vit_base_patch32_224', pretrained=True, num_classes=5)
model_tuned.to(device)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn

In [15]:
lr = 0.01
optimizer_tuned = optim.SGD(model_tuned.parameters(), lr=lr)
# scheduler = optim.lr_scheduler.StepLR(optimizer_tuned, step_size=5, gamma=0.5)
loss_function = nn.CrossEntropyLoss()
n_epochs = 20
train_loss_list = []
train_acc_list = []
valid_loss_list = []
valid_acc_list = []

for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    model_tuned.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]

        model_tuned.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer_tuned.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    model_tuned.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = model_tuned.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())
    
    train_loss_list.append(np.mean(losses_train))
    train_acc_list.append(acc_train/n_train)
    valid_loss_list.append(np.mean(losses_valid))
    valid_acc_list.append(acc_val/n_val)
    
#     scheduler.step()

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

EPOCH: 0, Train [Loss: 0.363, Accuracy: 0.873], Valid [Loss: 0.200, Accuracy: 0.928]
EPOCH: 1, Train [Loss: 0.123, Accuracy: 0.956], Valid [Loss: 0.205, Accuracy: 0.921]
EPOCH: 2, Train [Loss: 0.088, Accuracy: 0.970], Valid [Loss: 0.180, Accuracy: 0.934]
EPOCH: 3, Train [Loss: 0.066, Accuracy: 0.980], Valid [Loss: 0.170, Accuracy: 0.944]
EPOCH: 4, Train [Loss: 0.046, Accuracy: 0.987], Valid [Loss: 0.141, Accuracy: 0.947]
EPOCH: 5, Train [Loss: 0.035, Accuracy: 0.992], Valid [Loss: 0.139, Accuracy: 0.955]
EPOCH: 6, Train [Loss: 0.035, Accuracy: 0.991], Valid [Loss: 0.161, Accuracy: 0.939]
EPOCH: 7, Train [Loss: 0.029, Accuracy: 0.993], Valid [Loss: 0.149, Accuracy: 0.945]
EPOCH: 8, Train [Loss: 0.025, Accuracy: 0.994], Valid [Loss: 0.150, Accuracy: 0.947]
EPOCH: 9, Train [Loss: 0.023, Accuracy: 0.994], Valid [Loss: 0.162, Accuracy: 0.945]
EPOCH: 10, Train [Loss: 0.019, Accuracy: 0.995], Valid [Loss: 0.143, Accuracy: 0.957]
EPOCH: 11, Train [Loss: 0.019, Accuracy: 0.996], Valid [Loss: 0.

In [16]:
result_dict = base_dict.copy()
result_dict['model'] = [model_tuned]
result_dict['optimizer'] = optimizer_tuned
result_dict['loss_function'] = loss_function
result_dict['learning_rate'] = lr
result_dict['num_epoch'] = n_epochs

result_dict['min_train_loss__epoch'] = train_loss_list.index(min(train_loss_list))
result_dict['min_train_loss__train_loss'] = train_loss_list[train_loss_list.index(min(train_loss_list))]
result_dict['min_train_loss__train_acc'] = train_acc_list[train_loss_list.index(min(train_loss_list))]
result_dict['min_train_loss__valid_loss'] = valid_loss_list[train_loss_list.index(min(train_loss_list))]
result_dict['min_train_loss__valid_acc'] = valid_acc_list[train_loss_list.index(min(train_loss_list))]

result_dict['max_train_acc__epoch'] = train_acc_list.index(max(train_acc_list))
result_dict['max_train_acc__train_loss'] = train_loss_list[train_acc_list.index(max(train_acc_list))]
result_dict['max_train_acc__train_acc'] = train_acc_list[train_acc_list.index(max(train_acc_list))]
result_dict['max_train_acc__valid_loss'] = valid_loss_list[train_acc_list.index(max(train_acc_list))]
result_dict['max_train_acc__valid_acc'] = valid_acc_list[train_acc_list.index(max(train_acc_list))]

result_dict['min_valid_loss__epoch'] = valid_loss_list.index(min(valid_loss_list))
result_dict['min_valid_loss__train_loss'] = train_loss_list[valid_loss_list.index(min(valid_loss_list))]
result_dict['min_valid_loss__train_acc'] = train_acc_list[valid_loss_list.index(min(valid_loss_list))]
result_dict['min_valid_loss__valid_loss'] = valid_loss_list[valid_loss_list.index(min(valid_loss_list))]
result_dict['min_valid_loss__valid_acc'] = valid_acc_list[valid_loss_list.index(min(valid_loss_list))]

result_dict['max_valid_acc__epoch'] = valid_acc_list.index(max(valid_acc_list))
result_dict['max_valid_acc__train_loss'] = train_loss_list[valid_acc_list.index(max(valid_acc_list))]
result_dict['max_valid_acc__train_acc'] = train_acc_list[valid_acc_list.index(max(valid_acc_list))]
result_dict['max_valid_acc__valid_loss'] = valid_loss_list[valid_acc_list.index(max(valid_acc_list))]
result_dict['max_valid_acc__valid_acc'] = valid_acc_list[valid_acc_list.index(max(valid_acc_list))]

In [17]:
pd.DataFrame(result_dict)

,image_height,image_width,transform_train,transform_test,val_ratio,batch_size,model,optimizer,loss_function,learning_rate,...,min_valid_loss__epoch,min_valid_loss__train_loss,min_valid_loss__train_acc,min_valid_loss__valid_loss,min_valid_loss__valid_acc,max_valid_acc__epoch,max_valid_acc__train_loss,max_valid_acc__train_acc,max_valid_acc__valid_loss,max_valid_acc__valid_acc
0,224,224,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VisionTransformer(\n (patch_embed): PatchEmbe...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.005,...,5,0.034932,0.992435,0.139283,0.955331,10,0.01936,0.994957,0.142508,0.956772


In [18]:
result_df = pd.merge(result_df, pd.DataFrame(result_dict), how='outer')

In [19]:
# result_df.loc[13]['model']

In [20]:
# result_df.to_csv('homework2.csv')

In [21]:
# pd.read_csv('homework2.csv', index_col=0)

,image_height,image_width,transform_train,transform_test,val_ratio,batch_size,model,optimizer,loss_function,learning_rate,...,min_valid_loss__epoch,min_valid_loss__train_loss,min_valid_loss__train_acc,min_valid_loss__valid_loss,min_valid_loss__valid_acc,max_valid_acc__epoch,max_valid_acc__train_loss,max_valid_acc__train_acc,max_valid_acc__valid_loss,max_valid_acc__valid_acc
0,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.731455,0.720821,0.804275,0.726225,4,0.731455,0.720821,0.804275,0.726225
1,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.571860,0.778458,0.762222,0.716138,4,0.571860,0.778458,0.762222,0.716138
2,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.670148,0.737752,0.826592,0.691643,4,0.670148,0.737752,0.826592,0.691643
3,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.517022,0.805115,0.626215,0.766571,4,0.517022,0.805115,0.626215,0.766571
4,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.532323,0.801513,0.561638,0.793948,4,0.532323,0.801513,0.561638,0.793948
5,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,3,0.522402,0.804395,0.606840,0.795389,3,0.522402,0.804395,0.606840,0.795389
6,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,64,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.517843,0.806196,0.676056,0.740634,2,0.652513,0.761888,0.716556,0.746398
7,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,4,0.787554,0.702089,0.791451,0.713256,4,0.787554,0.702089,0.791451,0.713256
8,128,128,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(1...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,8,0.695362,0.737032,0.745436,0.717579,8,0.695362,0.737032,0.745436,0.717579
9,256,256,Compose(\n ToPILImage()\n RandomHorizont...,Compose(\n ToPILImage()\n Resize(size=(2...,0.2,32,VGG(\n (features): Sequential(\n (0): Conv...,SGD (\nParameter Group 0\n dampening: 0\n ...,CrossEntropyLoss(),0.0500,...,7,0.587865,0.780620,0.595602,0.779539,8,0.583386,0.777017,0.625318,0.785303


In [22]:
result_df.loc[39]

image_height                                                                224
image_width                                                                 224
transform_train               Compose(\n    ToPILImage()\n    RandomHorizont...
transform_test                Compose(\n    ToPILImage()\n    Resize(size=(2...
val_ratio                                                                   0.2
batch_size                                                                   32
model                         VisionTransformer(\n  (patch_embed): PatchEmbe...
optimizer                     SGD (\nParameter Group 0\n    dampening: 0\n  ...
loss_function                                                CrossEntropyLoss()
learning_rate                                                              0.01
num_epoch                                                                    20
min_train_loss__epoch                                                        19
min_train_loss__train_loss              

In [24]:
model_tuned.to(device)
model_tuned.eval()
t_pred = []
for x in dataloader_test:
    x = x.to(device)
    output = model_tuned(x)
    pred = output.argmax(1).tolist()
    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('/root/userspace/day2/homework2/submission2_pred_2.csv', header=True, index_label='id')

## アプローチ  
1. VGGで画像のトランスフォームやエポック数のテストを行い良さそうなパラメータを選ぶ
1. いろいろなモデルを試す　←　工夫した点　[timmドキュメント](https://rwightman.github.io/pytorch-image-models/)
1. よかったモデルで1.をもう一度行う
1. その時のパラメータで2.を行う
1. これを繰り返す...予定だったが思いのほか2.で時間がかかってしまったので3.以降はできていないです

## 気が付いた点？
* 転移学習は学習率をあらかじめ小さめに設定する方がよい？　→　さらにエポック数を大きめに設定する？　←　注意した点
* optimizerはAdamよりSGDのほうがいい結果になった　→　これはAdamで試した時の学習率が大きかったから？

## 疑問点
*  気が付いた点はそもそも正しい？
* 転移学習の過学習対策は？
    - 学習率を小さめに設定
    - schedulerの設定　←　こちらはあまりうまくいかず
* GPUのメモリに乗り切らないときの対処法
    - バッチサイズを小さくする
    - 学習のたびにカーネルを手動で一度リセットする

```sh
RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 11.17 GiB total capacity; 10.08 GiB already allocated; 15.44 MiB free; 10.85 GiB reserved in total by PyTorch)
```